In [1]:
import numpy as np
from src.var_processor.vpu import VPUBinaryZM
from src.tests.vpu_wrapper import VPUWrapper, signal_pre_processor

In [3]:
from pyinstrument import Profiler

profiler = Profiler()
profiler.start()

# code you want to profile ----- #
# Try four VPUs with L = 4
size = 4
residual_array = np.zeros(shape=(size, 100, size))
sum_err = np.zeros(shape=(size, 100))
for i in range(0, 100):
    # Initialise two VPUs and wrappers
    data_in = np.random.randint(255, size=(size, 1))
    mean = np.asarray([128]*size).reshape(-1, 1)
    vpus = [VPUBinaryZM(size) for _ in range(0, size)]
    wrappers = [VPUWrapper(vpu) for vpu in vpus]
    for _ in range(0, 1000):
        # First VPU
        input_signal = signal_pre_processor(data_in, mean)
        for w, wrapper in enumerate(wrappers):
            _, _, residual = wrapper.iterate(input_signal)
            residual_array[:, i, w] = residual.flatten()
            input_signal = residual
    pred_sum = 0
    for wrapper in wrappers:
        pred_sum
    pred_estimate_sum = np.asarray([wrap.pred_estimate for wrap in wrappers]).sum(axis=0)
    scale_factor = np.minimum(mean, 255-mean)
    est = (mean+pred_estimate_sum*scale_factor).astype(np.uint8)
    sum_err[:, i] = (data_in - est).flatten()
    print(f"Data: {data_in} - Estimate: {est}\n")
MSE = np.abs(sum_err.astype(np.int8)).mean(axis=1); MSE

# ----- #

profiler.stop()

print(profiler.output_text(unicode=True, color=True))

Data: [[ 92]
 [ 37]
 [145]
 [ 44]] - Estimate: [[ 92]
 [ 42]
 [162]
 [ 46]]

Data: [[207]
 [  6]
 [ 16]
 [  7]] - Estimate: [[215]
 [ 11]
 [ 16]
 [ 11]]

Data: [[172]
 [ 24]
 [129]
 [248]] - Estimate: [[170]
 [ 48]
 [128]
 [207]]

Data: [[ 39]
 [211]
 [192]
 [230]] - Estimate: [[ 46]
 [201]
 [171]
 [196]]

Data: [[ 40]
 [ 64]
 [114]
 [238]] - Estimate: [[ 26]
 [ 61]
 [113]
 [228]]

Data: [[226]
 [224]
 [122]
 [ 77]] - Estimate: [[221]
 [214]
 [120]
 [ 62]]

Data: [[241]
 [ 47]
 [226]
 [193]] - Estimate: [[231]
 [ 58]
 [239]
 [203]]

Data: [[  8]
 [132]
 [159]
 [  5]] - Estimate: [[ 15]
 [128]
 [150]
 [ 15]]

Data: [[  0]
 [229]
 [216]
 [209]] - Estimate: [[ 21]
 [235]
 [226]
 [221]]

Data: [[220]
 [214]
 [187]
 [177]] - Estimate: [[198]
 [199]
 [165]
 [189]]

Data: [[216]
 [ 90]
 [254]
 [145]] - Estimate: [[226]
 [ 94]
 [227]
 [158]]

Data: [[ 87]
 [  6]
 [100]
 [ 54]] - Estimate: [[113]
 [ 36]
 [124]
 [ 36]]

Data: [[200]
 [  5]
 [252]
 [240]] - Estimate: [[194]
 [ 15]
 [240]
 [237]]


Interesting:
* Adding to the array takes up about half the time. 
* Forward takes longer than backward.
* Linalg takes 4 seconds on each power iteration.
* The non_linearity is not needed on the wrapper.
* Update cov takes the same time as the backward and forward passes - we can half iteration time by not constantly updating the cov and the eigenvector.

In [ ]:
# Generate random length 4 array
rand_vec = np.random.uniform(size=(4, 1)); rand_vec

In [ ]:
%%timeit
a = np.sqrt((rand_vec**2).sum())

In [ ]:
%%timeit
b = np.abs(rand_vec).sum()

In [ ]:
%%timeit
c = np.linalg.norm(rand_vec)

In [ ]:
a = np.sqrt((rand_vec**2).sum())
b = np.abs(rand_vec).sum()
c = np.linalg.norm(rand_vec)
print(a, b, c)